# BigQuery - BQML

This notebook uses BigQuery ML to train a model to predict the value handwritten digits in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then evaluated and used for prediction using BigQuery directly.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-02.png">

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [1]:
%%bigquery
CREATE OR REPLACE MODEL `statmike-mlops.digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `statmike-mlops.digits.digits_prepped`
WHERE splits = 'TRAIN'

""


Review the iterations from training:

In [2]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `statmike-mlops.digits.digits_lr`)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.194654,0.194074,0.2,4688
1,0,1,0.141666,0.140428,0.4,6906
2,0,2,0.087523,0.086688,0.8,6480
3,0,3,0.052144,0.053070,1.6,6564
4,0,4,0.032740,0.034683,3.2,6148
5,0,5,0.021720,0.025581,6.4,6932
6,0,6,0.015503,0.019363,12.8,6663


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [3]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `statmike-mlops.digits.digits_lr`,
    (SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `statmike-mlops.digits.digits_lr`,
    (SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TEST'))

,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TEST,0.944906,0.941761,0.942779,0.941532,0.212325,0.998918
1,TRAIN,0.963700,0.963412,0.962937,0.963276,0.161991,0.999120


Review the confusion matrix for each split:

In [4]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,136,0,0,0,0,0,1,0,0,0
1,1,0,134,1,0,0,0,0,0,3,7
2,2,0,0,144,0,0,0,0,0,0,0
3,3,1,0,3,143,0,1,0,0,4,2
4,4,0,1,0,0,136,0,0,1,3,0
5,5,0,0,0,0,0,138,1,0,0,4
6,6,0,1,0,0,0,0,134,0,0,0
7,7,0,0,0,0,1,0,0,143,0,1
8,8,0,5,0,0,0,3,1,0,129,2
9,9,0,0,0,1,0,3,0,0,2,140


In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`
    WHERE splits = 'TEST')
  );

,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,40,0,0,0,1,0,0,0,0,0
1,1,0,32,0,0,0,0,0,0,3,2
2,2,0,0,33,0,0,0,0,0,0,0
3,3,0,0,0,27,0,0,0,1,0,1
4,4,0,0,0,0,37,0,0,2,0,1
5,5,0,0,0,0,1,37,0,0,0,1
6,6,0,0,0,0,1,0,45,0,0,0
7,7,0,0,0,0,0,0,0,32,1,1
8,8,0,1,1,0,0,0,0,0,29,3
9,9,0,0,0,0,0,0,0,0,0,34


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [6]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`
    WHERE splits='TEST')
  )

Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [9]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'int64_field_0', 'p0',
       'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11',
       'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21',
       'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31',
       'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41',
       'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51',
       'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61',
       'p62', 'p63', 'target', 'target_OE', 'SPLITS'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [11]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,9,9,"[{'label': 9, 'prob': 0.9495711090506438}, {'l...",TEST
1,9,9,"[{'label': 9, 'prob': 0.9882851618911664}, {'l...",TEST
2,2,2,"[{'label': 2, 'prob': 0.9462652018611037}, {'l...",TEST
3,2,2,"[{'label': 2, 'prob': 0.8908876430057159}, {'l...",TEST
4,1,1,"[{'label': 1, 'prob': 0.9404011917883357}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [12]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 0.0033453333504508435},
  {'label': 1, 'prob': 6.498440987206625e-05},
  {'label': 2, 'prob': 0.0013975646580931275},
  {'label': 3, 'prob': 0.028135989150226727},
  {'label': 4, 'prob': 4.0994317726569685e-05},
  {'label': 5, 'prob': 0.01680373105007372},
  {'label': 6, 'prob': 3.640815233225878e-05},
  {'label': 7, 'prob': 0.0003050433397378075},
  {'label': 8, 'prob': 0.0002988425208431634},
  {'label': 9, 'prob': 0.9495711090506438}]]

---
## Remove Resources
- delete model `statmike-mlops.digits.digits_lr`

In [ ]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
bq.delete_model('statmike-mlops.digits.digits_lr')